## 모듈

In [ ]:
!pip3 install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime
import pickle
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/날씨')
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import itertools

import pandas as pd
import pandas_profiling
import numpy as np
import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid' )
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
pyo.init_notebook_mode()
#import scikitplot

from scipy import stats


import sklearn
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import decomposition
from sklearn import metrics
from sklearn.utils.fixes import loguniform
from sklearn import model_selection
from sklearn import ensemble
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import optuna
from optuna.samplers import TPESampler

import xgboost as xgb
import lightgbm as lgb 
import xgboost as xgb
#import catboost as cb
import statsmodels.api as sm


Output hidden; open in https://colab.research.google.com to view.

## load data

In [ ]:
with open("/content/drive/MyDrive/날씨/lgb_df.pkl","rb") as fr:
    data = pickle.load(fr)
df = data.copy()

In [ ]:
df['year'] = np.where(df.index.year==2020,0,
                      np.where(df.index.year==2021,1,2))

In [ ]:
kfcv = model_selection.KFold(10,shuffle=True,random_state=42)

### 함수

In [ ]:
def  stn_split(df:pd.DataFrame,spn_num :int)->pd.DataFrame:
  if 'stn' in df.columns:
    df_stn_num = df.loc[df['stn']==spn_num,:]
    df_stn_num = df_stn_num.drop(['stn'],axis=1)
    return df_stn_num  

def tr_ts_split(df:pd.DataFrame)->(pd.DataFrame,pd.DataFrame):
  train_df = df.loc[(df.index<'2021-06')| (df.index>='2021-07-01'),:]
  test_df = df.loc[(df.index>='2021-06')&(df.index<'2021-07'),:]
  return train_df,test_df


def stationarity_test(df:pd.Series)-> (pd.DataFrame,pd.DataFrame):
  Stationarity_kpss = sm.tsa.stattools.kpss(df)
  Stationarity_kpss_df = pd.Series(Stationarity_kpss[0:3],
                                   index=['Test Statistics', 'p-value', 'Used Lag'])
  for key, value in Stationarity_kpss[3].items():
    Stationarity_kpss_df[f'Critical Value {key}'] = value
  Stationarity_kpss_df = pd.DataFrame(Stationarity_kpss_df, columns=['Stationarity_kpss'])
  
  Stationarity_adf = sm.tsa.stattools.adfuller(df)
  Stationarity_adf_df = pd.Series(Stationarity_adf[0:4],
                                  index=['Test Statistics', 'p-value', 'Used Lag', 'Used Observations'])
  for key, value in Stationarity_adf[4].items():
    Stationarity_adf_df[f'Critical Value {key}'] = value
  Stationarity_adf_df['Maximum Information Criteria'] = Stationarity_adf[5]
  Stationarity_adf_df = pd.DataFrame(Stationarity_adf_df, columns=['Stationarity_adf'])

  return pd.merge(Stationarity_kpss_df, Stationarity_adf_df, left_index=True,  right_index = True , how ='outer')


def time_feature(df:pd.DataFrame)->pd.DataFrame:
  x_list = ['solarza','esr','band1','band2','band3','band4','band6','band7','band12']
  df_feature = df.copy()
  for x in x_list:
    for n in [1,144]:
      if f'{x}_{n}' not in df_feature.columns:
        df_feature[f'{x}_{n}'] = df_feature[x].diff(n).bfill()
    for n in [72,144]:
      if f'{x}_{n}ma' not in df_feature.columns:
        df_feature[f'{x}_{n}ma'] = df_feature[x].rolling(n).mean().bfill()
  return df_feature

## 튜닝

In [ ]:
df_146 = stn_split(df,146)

In [ ]:
df_146 = time_feature(df_146)
df_146.columns

Index(['uv', 'band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7',
       'band8', 'band9', 'band10', 'band11', 'band12', 'band13', 'band14',
       'band15', 'band16', 'solarza', 'esr', 'Month sin', 'Month cos',
       'Day sin', 'Day cos', '2/Day sin', '2/Day cos', '3/Day sin',
       '3/Day cos', 'year', 'solarza_1', 'solarza_144', 'solarza_72ma',
       'solarza_144ma', 'esr_1', 'esr_144', 'esr_72ma', 'esr_144ma', 'band1_1',
       'band1_144', 'band1_72ma', 'band1_144ma', 'band2_1', 'band2_144',
       'band2_72ma', 'band2_144ma', 'band3_1', 'band3_144', 'band3_72ma',
       'band3_144ma', 'band4_1', 'band4_144', 'band4_72ma', 'band4_144ma',
       'band6_1', 'band6_144', 'band6_72ma', 'band6_144ma', 'band7_1',
       'band7_144', 'band7_72ma', 'band7_144ma', 'band12_1', 'band12_144',
       'band12_72ma', 'band12_144ma'],
      dtype='object')

In [ ]:
def tr_ts_split(df:pd.DataFrame)->(pd.DataFrame,pd.DataFrame):
  train_df = df.loc[(df.index<'2021-06')| (df.index>='2021-07-01'),:]
  test_df = df.loc[(df.index>='2021-06')&(df.index<'2021-07'),:]
  return train_df,test_df

### LGBM

In [ ]:
lgbr=  lgb.LGBMRegressor(n_jobs=-1,randoms_state=42)

In [ ]:
lgbr_rand_params = {
    'num_leaves':randint(10,100),
    'max_depth':randint(1,10),
    'min_child_samples':randint(10,40),
    'n_estimators':randint(50,300),
    'learning_rate' : [0.5,0.1]
#    'subsample' : loguniform(0.6,1.0),
#    'subsample_feq':randint(1,5)
#		'colsample_bytree':logunifomr(0.65,1)
}

In [ ]:
rand_lgbr = model_selection.RandomizedSearchCV(lgbr ,
                                              lgbr_params,
                                              n_iter=500,
                                              cv=kfcv,
                                              random_state=42,
                                              scoring ='neg_root_mean_squared_error',
                                              return_train_score=True,
                                              n_jobs=-1)

In [ ]:
rand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=LGBMRegressor(randoms_state=42), n_iter=500,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.5, 0.1],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff563882b10>,
                                        'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff56389f790>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff56389f690>,
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff563882f90>},
                   random_state=42, return_train_score=True,
                   scoring='neg_root_mean_squared_error')

In [ ]:
rand_lgbr.best_estimator_

LGBMRegressor(max_depth=9, min_child_samples=39, n_estimators=295,
              num_leaves=69, randoms_state=42)

In [ ]:
rand_lgbr.best_score_

-0.23812686879610254

In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[293,295,297],
    'learning_rate' : [0.05,0.1],
    'subsample' : [0.6,0.8,1],
    'subsample_feq':[1,3,5],
		'colsample_bytree':[0.6,0.8,1]
}

In [ ]:
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

In [ ]:
grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMRegressor(randoms_state=42), n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1],
                         'learning_rate': [0.05, 0.1], 'max_depth': [9],
                         'min_child_samples': [39],
                         'n_estimators': [293, 295, 297], 'num_leaves': [69],
                         'subsample': [0.6, 0.8, 1],
                         'subsample_feq': [1, 3, 5]},
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [ ]:
grand_lgbr.best_estimator_

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=297, num_leaves=69, randoms_state=42, subsample=0.6,
              subsample_feq=1)

In [ ]:
grand_lgbr.best_score_

-0.23334293247085433

In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[300,320,330],
    'learning_rate' : [0.07,0.1],
    'subsample' : [0.5,0.4],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}

In [ ]:
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

In [ ]:
grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(grand_lgbr.best_estimator_ ,grand_lgbr.best_score_)

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=330, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1) -0.23218305113268997


In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[330,400],
    'learning_rate' : [0.07,0.1],
    'subsample' : [0.5,0.45],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}

In [ ]:
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

In [ ]:
grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(grand_lgbr.best_estimator_ ,grand_lgbr.best_score_)

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=400, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1) -0.2304147487779033


In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[500,700],
    'learning_rate' : [0.07,0.1],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(grand_lgbr.best_estimator_ ,grand_lgbr.best_score_)

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=700, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1) -0.22667189974225854


In [ ]:
grand_lgbr.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMRegressor(randoms_state=42), n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'learning_rate': [0.07, 0.1],
                         'max_depth': [9], 'min_child_samples': [39],
                         'n_estimators': [500, 700], 'num_leaves': [69],
                         'subsample': [0.5], 'subsample_feq': [1]},
             return_train_score=True, scoring='neg_root_mean_squared_error')>

In [ ]:
hlgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[700,900],
    'learning_rate' : [0.07,0.1],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(grand_lgbr.best_estimator_ ,grand_lgbr.best_score_)

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=700, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1) -0.22667189974225854


In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[700,670,730],
    'learning_rate' : [0.1],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=700, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1),
-0.22667189974225854


In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'n_estimators':[695,703,700],
    'learning_rate' : [0.1],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

In [ ]:
lgbr = grand_lgbr.best_estimator_

In [ ]:
grand_lgbr.best_estimator_

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              n_estimators=700, num_leaves=69, randoms_state=42, subsample=0.5,
              subsample_feq=1)

In [ ]:
y_pred =lgbr.predict(test_108.iloc[:,1:])

In [ ]:
metrics.mean_squared_error(y_pred,test_108['uv'])**0.5

0.4957577192818956

In [ ]:
lgbr_grid_params = {
    'num_leaves':[69],
    'max_depth':[9],
    'min_child_samples':[39],
    'min_split_gain' : [0,0.1],
    'n_estimators':[695,703,700],
    'learning_rate' : [0.1,0.2],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1],
    'reg_alpha' : [0,0.1],
    'reg_lambda' : [0,0.1]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, max_depth=9, min_child_samples=39,
              min_split_gain=0, n_estimators=703, num_leaves=69,
              randoms_state=42, reg_alpha=0.1, reg_lambda=0, subsample=0.5,
              subsample_feq=1),
-0.22624516706931835


In [ ]:
lgbr_grid_params = {
    'num_leaves':[73,70,76],
    'max_depth':[10,12,15],
    'min_child_samples':[39],
    'min_split_gain' : [0],
    'n_estimators':[715,720,745],
    'learning_rate' : [0.095,0.09],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1],
    'reg_alpha' : [0.1],
    'reg_lambda' : [0]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, learning_rate=0.09, max_depth=15,
              min_child_samples=39, min_split_gain=0, n_estimators=745,
              num_leaves=76, randoms_state=42, reg_alpha=0.1, reg_lambda=0,
              subsample=0.5, subsample_feq=1),
-0.22345419408026093


In [ ]:
lgbr_grid_params = {
    'num_leaves':[78,86],
    'max_depth':[15,17],
    'min_child_samples':[39],
    'min_split_gain' : [0],
    'n_estimators':[755,800],
    'learning_rate' : [0.09,0.08],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1],
    'reg_alpha' : [0.1],
    'reg_lambda' : [0]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, learning_rate=0.09, max_depth=15,
              min_child_samples=39, min_split_gain=0, n_estimators=800,
              num_leaves=78, randoms_state=42, reg_alpha=0.1, reg_lambda=0,
              subsample=0.5, subsample_feq=1),
-0.22244070979792263


In [ ]:
lgbr_grid_params = {
    'num_leaves':[78,80],
    'max_depth':[15,17],
    'min_child_samples':[39,40],
    'min_split_gain' : [0],
    'n_estimators':[800,850,900],
    'learning_rate' : [0.09,0.085],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1],
    'reg_alpha' : [0.1],
    'reg_lambda' : [0]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, learning_rate=0.09, max_depth=15,
              min_child_samples=39, min_split_gain=0, n_estimators=900,
              num_leaves=78, randoms_state=42, reg_alpha=0.1, reg_lambda=0,
              subsample=0.5, subsample_feq=1),
-0.22201293805473607


In [ ]:
lgbr = grand_lgbr.best_estimator_

In [ ]:
lgbr.fit(train_108.iloc[:,1:],train_108['uv'])

LGBMRegressor(colsample_bytree=1, learning_rate=0.09, max_depth=15,
              min_child_samples=39, min_split_gain=0, n_estimators=900,
              num_leaves=78, randoms_state=42, reg_alpha=0.1, reg_lambda=0,
              subsample=0.5, subsample_feq=1)

In [ ]:
y_pred =lgbr.predict(test_108.iloc[:,1:])
metrics.mean_squared_error(y_pred,test_108['uv'])**0.5

0.498519519946119

In [ ]:
lgbr_grid_params = {
    'num_leaves':[78],
    'max_depth':[15],
    'min_child_samples':[39],
    'min_split_gain' : [0],
    'n_estimators':[900,1200],
    'learning_rate' : [0.09],
    'subsample' : [0.5],
    'subsample_feq':[1],
		'colsample_bytree':[1],
    'reg_alpha' : [0.1],
    'reg_lambda' : [0]
}
grand_lgbr = model_selection.GridSearchCV(lgbr,
                                          lgbr_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_lgbr.fit(train_108.iloc[:,1:],train_108['uv'])
print(f'{grand_lgbr.best_estimator_},\n{grand_lgbr.best_score_}')

LGBMRegressor(colsample_bytree=1, learning_rate=0.09, max_depth=15,
              min_child_samples=39, min_split_gain=0, n_estimators=1200,
              num_leaves=78, randoms_state=42, reg_alpha=0.1, reg_lambda=0,
              subsample=0.5, subsample_feq=1),
-0.2212412121504475


In [ ]:
X = df_108.iloc[:,1:]
y = df_108['uv']
_n_trails = 100

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params

def lgbr_objective(trial):
    lgbr_param = {
          'n_estimators' : trial.suggest_int('n_estimators',500,1500),
          'max_depth' : trial.suggest_int('max_depth',7,35),
          'colsample_bytree': trial.suggest_float('colsample_bytree',0.9,1.0),
          'learning_rate' : trial.suggest_loguniform('learning_rate',0.085,0.2),
          'min_child_samples' : trial.suggest_int('min_child_samples',37,45),
          'min_split_gain': trial.suggest_loguniform('min_split_gain',1e-16,0.02),
          'num_leaves' : trial.suggest_int('num_leaves',60,90),
          'reg_alpha' : trial.suggest_loguniform('reg_alpha',0.05,2.0),
          'reg_lambda' : trial.suggest_loguniform('reg_lambda',0.0001,0.1),
          'subsample' : trial.suggest_float('subsample',0.4,1.0),
          'subsample_freq' : trial.suggest_int('subsample_freq',0,3)

    }



    lgbr=  lgb.LGBMRegressor(n_jobs=-1,randoms_state=42,**lgbr_param)
    
    
    scores = model_selection.cross_val_score(lgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error"
                             )
    return scores.mean()

In [ ]:
lgbr_params = tune(lgbr_objective)

[I 2022-07-31 06:47:44,598] A new study created in memory with name: no-name-02687665-931b-4937-9478-9764c26b8fa9
[I 2022-07-31 06:49:51,485] Trial 0 finished with value: -0.23004022542223743 and parameters: {'n_estimators': 874, 'max_depth': 34, 'colsample_bytree': 0.9731993941811405, 'learning_rate': 0.14186884909739528, 'min_child_samples': 38, 'min_split_gain': 1.701697318427598e-14, 'num_leaves': 61, 'reg_alpha': 1.2207764786954152, 'reg_lambda': 0.006358358856676255, 'subsample': 0.8248435466776274, 'subsample_freq': 0}. Best is trial 0 with value: -0.23004022542223743.
[I 2022-07-31 06:53:32,149] Trial 1 finished with value: -0.22721563555556895 and parameters: {'n_estimators': 1470, 'max_depth': 31, 'colsample_bytree': 0.9212339110678276, 'learning_rate': 0.09930865415766783, 'min_child_samples': 38, 'min_split_gain': 2.2437653783828104e-12, 'num_leaves': 76, 'reg_alpha': 0.24602080610141613, 'reg_lambda': 0.0007476312062252305, 'subsample': 0.7671117368334277, 'subsample_freq'

KeyboardInterrupt: ignored

In [ ]:
lgb_parameters = {'n_estimators': 1181,
                  'max_depth': 21, 
                  'colsample_bytree': 0.9900654712519903,
                  'learning_rate': 0.09205453627834828,
                  'min_child_samples': 44, 
                  'min_split_gain': 3.171414772101719e-05, 
                  'num_leaves': 75, 'reg_alpha': 1.3736321906283118, 
                  'reg_lambda': 0.00019783308128221966, 
                  'subsample': 0.9100767985669579, 'subsample_freq': 1}

### XGB

In [ ]:
xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror')

In [ ]:
xgb_grid_params = {
                  'booster' : ['dart','gblinear','gbtree']
                  }
grand_xgbr = model_selection.GridSearchCV(xgbr,
                                          xgb_grid_params,
                                          cv=kfcv,
                                          scoring ='neg_root_mean_squared_error',
                                          return_train_score=True,
                                          n_jobs=-1)

grand_xgbr.fit(df_108.iloc[:,1:],df_108['uv'])
print(f'{grand_xgbr.best_estimator_},\n{grand_xgbr.best_score_}')

XGBRegressor(booster='gblinear', n_jobs=-1, objective='reg:squarederror',
             tree_method='gpu_hist'),
-1.4147727665867977


In [ ]:
xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state =42)

In [ ]:
#!pip install optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
X = df_108.iloc[:,1:]
y = df_108['uv']
_n_trails = 100

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study

def xgb_objective(trial):
    list_bins = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250,300]
    _grow_policy = trial.suggest_categorical('grow_policy',['depthwise','lossguide']) 
    _n_estimators = trial.suggest_int('n_estimators',300,1200)
    _max_depth = trial.suggest_int('max_depth', 3, 20)    
    _max_leaves = trial.suggest_int('max_leaves',0,255)                  
    _learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.8)
    _gamma =  trial.suggest_loguniform('gamma',0.01,3.0)
    _min_child_weight =trial.suggest_categorical('min_child_weight',[1,2,3,4,5,6,7,8,9])
    _subsample = trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.1)
    _colsample_bytree=trial.suggest_discrete_uniform('colsample_bytree',0.5,1.0,0.05)
    _colsample_bylevel=trial.suggest_discrete_uniform('colsample_bylevel',0.5,1.0,0.05)    
    _colsample_bynode=trial.suggest_discrete_uniform('colsample_bynode',0.5,1.0,0.05)    
    _reg_lambda = trial.suggest_int('reg_lambda', 1, 6)
    _reg_alpha = trial.suggest_int('reg_alpha', 0, 3)
    _max_bin = trial.suggest_categorical('max_bin', list_bins)
    _sample_type =  trial.suggest_categorical('sample_type',['uniform','weighted'])
    _nomarlize_type = trial.suggest_categorical('nomarlize_type',['tree','forest'])
    _rate_drop = trial.suggest_float('rate_drop',0.0,1.0)
    _one_drop = trial.suggest_categorical('one_drop',[0,1])
    _skip_drop = trial.suggest_float('skip_drop',0.0,0.6)



    xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state =42,
                        grow_policy = _grow_policy,
                        n_estimator = _n_estimators,
                        max_depth = _max_depth,
                        learning_rate = _learning_rate,
                        gamma = _gamma,
                        min_child_weight = _min_child_weight,
                        subsample = _subsample,
                        colsample_bytree = _colsample_bytree,
                        colsample_bylevel = _colsample_bylevel,
                        colsample_bynode = _colsample_bynode,
                        reg_lambda = _reg_lambda,
                        reg_alpha = _reg_alpha,
                        max_bin = _max_bin,
                        sample_type = _sample_type,
                        nomarlize_type = _nomarlize_type,
                        rate_drop = _rate_drop,
                        one_drop = _one_drop,
                        skip_drop = _skip_drop
                         )
    
    _fit_params={'early_stopping_rounds':100,
                'eval_set': [(X,y)],
                'eval_metric' : 'rmse',
                'verbose':0}
    
    scores = cross_val_score(xgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error",fit_params = _fit_params
                             )
    return scores.mean()

In [ ]:
xgbr_params = tune(xgb_objective)

[I 2022-07-30 16:13:22,322] A new study created in memory with name: no-name-e40842fd-917e-46cd-a8e3-88e42e25d7b7
[I 2022-07-30 16:15:00,389] Trial 0 finished with value: -0.24517257816770283 and parameters: {'grow_policy': 'lossguide', 'n_estimators': 959, 'max_depth': 13, 'max_leaves': 39, 'learning_rate': 0.1332356710656001, 'gamma': 0.013927723945289024, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 0.75, 'colsample_bylevel': 0.7, 'colsample_bynode': 0.65, 'reg_lambda': 4, 'reg_alpha': 0, 'max_bin': 100, 'sample_type': 'weighted', 'nomarlize_type': 'tree', 'rate_drop': 0.09767211400638387, 'one_drop': 0, 'skip_drop': 0.07322294090686729}. Best is trial 0 with value: -0.24517257816770283.
[I 2022-07-30 16:15:13,420] Trial 1 finished with value: -0.27224925927694577 and parameters: {'grow_policy': 'depthwise', 'n_estimators': 1119, 'max_depth': 7, 'max_leaves': 169, 'learning_rate': 0.25625175011063467, 'gamma': 0.19420998251718036, 'min_child_weight': 3, 'subsample': 

Best score: -0.24007365575489073

Optimized parameters: {'grow_policy': 'depthwise', 'n_estimators': 873, 'max_depth': 19, 'max_leaves': 13, 'learning_rate': 0.08834615853128412, 'gamma': 0.1179063358441864, 'min_child_weight': 9, 'subsample': 0.8, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.65, 'colsample_bynode': 0.7, 'reg_lambda': 3, 'reg_alpha': 0, 'max_bin': 75, 'sample_type': 'weighted', 'nomarlize_type': 'tree', 'rate_drop': 0.689790493226518, 'one_drop': 1, 'skip_drop': 0.34570580184187344}



In [ ]:
xgbr_params

{'colsample_bylevel': 0.65,
 'colsample_bynode': 0.7,
 'colsample_bytree': 0.9,
 'gamma': 0.1179063358441864,
 'grow_policy': 'depthwise',
 'learning_rate': 0.08834615853128412,
 'max_bin': 75,
 'max_depth': 19,
 'max_leaves': 13,
 'min_child_weight': 9,
 'n_estimators': 873,
 'nomarlize_type': 'tree',
 'one_drop': 1,
 'rate_drop': 0.689790493226518,
 'reg_alpha': 0,
 'reg_lambda': 3,
 'sample_type': 'weighted',
 'skip_drop': 0.34570580184187344,
 'subsample': 0.8}

In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
X = df_108.iloc[:,1:]
y = df_108['uv']
_n_trails = 40

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study

def xgb_objective(trial):
    _grow_policy = trial.suggest_categorical('grow_policy',['depthwise','lossguide']) 
    _n_estimators = trial.suggest_int('n_estimators',700,1000)
    _max_depth = trial.suggest_int('max_depth', 15, 30)    
    _max_leaves = trial.suggest_int('max_leaves',20,80)                  
    _learning_rate = trial.suggest_uniform('learning_rate', 0.07, 0.15)
    _gamma =  trial.suggest_loguniform('gamma',0.01,3.0)
    _min_child_weight =trial.suggest_categorical('min_child_weight',[5,6,7,8,9])
    _subsample = trial.suggest_float('subsample', 0.7, 0.9)
    _colsample_bytree=trial.suggest_float('colsample_bytree',0.8,1.0)
    _colsample_bylevel=trial.suggest_float('colsample_bylevel',0.65,0.7)    
    _colsample_bynode=trial.suggest_float('colsample_bynode',0.7,1.0) 
    _reg_lambda = trial.suggest_int('reg_lambda', 1, 5)
    _reg_alpha = trial.suggest_int('reg_alpha', 0, 2)
    _max_bin = trial.suggest_int('max_bin', 75,150)
    _sample_type =  trial.suggest_categorical('sample_type',['uniform','weighted'])
#    _nomarlize_type = trial.suggest_categorical('nomarlize_type',['tree'])
    _rate_drop = trial.suggest_float('rate_drop',0.0,1.0)
    _one_drop = trial.suggest_categorical('one_drop',[0,1])
    _skip_drop = trial.suggest_float('skip_drop',0.25,0.45)



    xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state =42,
                        normalize_type = 'tree',
                        max_leaves = _max_leaves,
                        grow_policy = _grow_policy,
                        n_estimator = _n_estimators,
                        max_depth = _max_depth,
                        learning_rate = _learning_rate,
                        gamma = _gamma,
                        min_child_weight = _min_child_weight,
                        subsample = _subsample,
                        colsample_bytree = _colsample_bytree,
                        colsample_bylevel = _colsample_bylevel,
                        colsample_bynode = _colsample_bynode,
                        reg_lambda = _reg_lambda,
                        reg_alpha = _reg_alpha,
                        max_bin = _max_bin,
                        sample_type = _sample_type,
                        rate_drop = _rate_drop,
                        one_drop = _one_drop,
                        skip_drop = _skip_drop
                         )
    
    _fit_params={'early_stopping_rounds':100,
                'eval_set': [(X,y)],
                'eval_metric' : 'rmse',
                'verbose':0}
    
    scores = cross_val_score(xgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error",fit_params = _fit_params
                             )
    return scores.mean()

In [ ]:
xgbr_params,xgb_study = tune(xgb_objective)

[I 2022-07-31 03:49:07,336] A new study created in memory with name: no-name-d290c486-caf0-4013-a83e-2ac7096ab744
[I 2022-07-31 03:49:17,736] Trial 0 finished with value: -0.2900791476044537 and parameters: {'grow_policy': 'lossguide', 'n_estimators': 920, 'max_depth': 24, 'max_leaves': 29, 'learning_rate': 0.08247956162689622, 'gamma': 0.013927723945289024, 'min_child_weight': 9, 'subsample': 0.8664885281600844, 'colsample_bytree': 0.8424678221356553, 'colsample_bylevel': 0.659091248360355, 'colsample_bynode': 0.7550213529560301, 'reg_lambda': 2, 'reg_alpha': 1, 'max_bin': 107, 'sample_type': 'weighted', 'rate_drop': 0.13949386065204183, 'one_drop': 1, 'skip_drop': 0.3412139968434072}. Best is trial 0 with value: -0.2900791476044537.
[I 2022-07-31 03:49:27,498] Trial 1 finished with value: -0.2996413652062452 and parameters: {'grow_policy': 'depthwise', 'n_estimators': 854, 'max_depth': 24, 'max_leaves': 22, 'learning_rate': 0.11860358815211507, 'gamma': 0.02644898018392711, 'min_chil

Best score: -0.2539969486500743

Optimized parameters: {'grow_policy': 'lossguide', 'n_estimators': 805, 'max_depth': 19, 'max_leaves': 76, 'learning_rate': 0.13941976428353292, 'gamma': 0.24956464995766725, 'min_child_weight': 8, 'subsample': 0.774115594214795, 'colsample_bytree': 0.9196362430536763, 'colsample_bylevel': 0.6669047053613084, 'colsample_bynode': 0.968735940586474, 'reg_lambda': 5, 'reg_alpha': 0, 'max_bin': 107, 'sample_type': 'weighted', 'rate_drop': 0.861545667508371, 'one_drop': 0, 'skip_drop': 0.25069190807533676}



In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go

In [ ]:
%matplotlib inline
pyo.init_notebook_mode()
optuna.visualization.matplotlib.plot_param_importances(xgb_study)

Output hidden; open in https://colab.research.google.com to view.

### XGG , MONTH [5,6,7,8]

In [ ]:
kfcv = model_selection.KFold(n_splits=3, shuffle=False)
df_146 = df_146.loc[df_146.index.month.isin([5,6,7,8]),:]

In [ ]:
X = df_146.iloc[:,1:]
y = df_146['uv']
_n_trails = 300

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study

def xgb_objective(trial):
    _grow_policy = trial.suggest_categorical('grow_policy',['depthwise','lossguide']) 
    _n_estimators = trial.suggest_int('n_estimators',300,3000)
    _max_depth = trial.suggest_int('max_depth', 1, 99)    
    _max_leaves = trial.suggest_int('max_leaves',20,80)                  
    _learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.15)
    _gamma =  trial.suggest_loguniform('gamma',0.01,3.0)
    _min_child_weight =trial.suggest_categorical('min_child_weight',[1,2,3,4,5,6,7,8,9])
    _subsample = trial.suggest_float('subsample', 0.7, 0.9)
    _colsample_bytree=trial.suggest_float('colsample_bytree',0.8,1.0)
    _colsample_bylevel=trial.suggest_float('colsample_bylevel',0.65,0.7)    
    _colsample_bynode=trial.suggest_float('colsample_bynode',0.7,1.0) 
    _reg_lambda = trial.suggest_int('reg_lambda', 1, 5)
    _reg_alpha = trial.suggest_int('reg_alpha', 0, 2)
    _max_bin = trial.suggest_int('max_bin', 75,150)
    _sample_type =  trial.suggest_categorical('sample_type',['uniform','weighted'])
#    _nomarlize_type = trial.suggest_categorical('nomarlize_type',['tree'])
    _rate_drop = trial.suggest_float('rate_drop',0.0,1.0)
    _one_drop = trial.suggest_categorical('one_drop',[0,1])
    _skip_drop = trial.suggest_float('skip_drop',0.25,0.45)



    xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state =42,
                        normalize_type = 'tree',
                        max_leaves = _max_leaves,
                        grow_policy = _grow_policy,
                        n_estimator = _n_estimators,
                        max_depth = _max_depth,
                        learning_rate = _learning_rate,
                        gamma = _gamma,
                        min_child_weight = _min_child_weight,
                        subsample = _subsample,
                        colsample_bytree = _colsample_bytree,
                        colsample_bylevel = _colsample_bylevel,
                        colsample_bynode = _colsample_bynode,
                        reg_lambda = _reg_lambda,
                        reg_alpha = _reg_alpha,
                        max_bin = _max_bin,
                        sample_type = _sample_type,
                        rate_drop = _rate_drop,
                        one_drop = _one_drop,
                        skip_drop = _skip_drop
                         )
    
    _fit_params={'early_stopping_rounds':100,
                'eval_set': [(X,y)],
                'eval_metric' : 'rmse',
                'verbose':0}
    
    scores = cross_val_score(xgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error",fit_params = _fit_params
                             )
    return scores.mean()

In [ ]:
xgbr_params,xgb_study = tune(xgb_objective)

[I 2022-08-01 08:00:47,652] A new study created in memory with name: no-name-debeb8eb-3b10-49ee-a520-26b444e712c1
[I 2022-08-01 08:00:55,190] Trial 0 finished with value: -2.0208239537267607 and parameters: {'grow_policy': 'lossguide', 'n_estimators': 2277, 'max_depth': 60, 'max_leaves': 29, 'learning_rate': 0.002185034568578638, 'gamma': 0.013927723945289024, 'min_child_weight': 5, 'subsample': 0.7608484485919075, 'colsample_bytree': 0.9049512863264476, 'colsample_bylevel': 0.6715972509321058, 'colsample_bynode': 0.7873687420594125, 'reg_lambda': 4, 'reg_alpha': 0, 'max_bin': 97, 'sample_type': 'weighted', 'rate_drop': 0.7851759613930136, 'one_drop': 1, 'skip_drop': 0.3684829137724085}. Best is trial 0 with value: -2.0208239537267607.
[I 2022-08-01 08:01:05,669] Trial 1 finished with value: -0.5092165647868666 and parameters: {'grow_policy': 'lossguide', 'n_estimators': 760, 'max_depth': 7, 'max_leaves': 77, 'learning_rate': 0.12627094381908754, 'gamma': 1.0057690178153986, 'min_child

Best score: -0.4726382464255844

Optimized parameters: {'grow_policy': 'lossguide', 'n_estimators': 1570, 'max_depth': 95, 'max_leaves': 76, 'learning_rate': 0.1493922711356767, 'gamma': 0.11079073578898974, 'min_child_weight': 4, 'subsample': 0.8954837522622376, 'colsample_bytree': 0.9243106076919249, 'colsample_bylevel': 0.6686071550993263, 'colsample_bynode': 0.87542298251727, 'reg_lambda': 5, 'reg_alpha': 0, 'max_bin': 135, 'sample_type': 'weighted', 'rate_drop': 0.963902626727559, 'one_drop': 1, 'skip_drop': 0.3880536098345327}



### GBLINEAR

In [ ]:
#!pip3 install gputill
'''
import GPUtil
from threading import Thread
import time

class Monitor(Thread):
    def __init__(self, delay):
        super(Monitor, self).__init__()
        self.stopped = False
        self.delay = delay # Time between calls to GPUtil
        self.start()

    def run(self):
        while not self.stopped:
            GPUtil.showUtilization()
            time.sleep(self.delay)

    def stop(self):
        self.stopped = True
        
monitor = Monitor(300)'''

In [ ]:
X = df_146.iloc[:,1:]
y = df_146['uv']
_n_trails = 200

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study

def xgb_objective(trial):
  
    xgb_param = {
          'n_estimators' : trial.suggest_int('n_estimators',100,5000),
          'learning_rate' : trial.suggest_loguniform('learning_rate',0.001,0.8),
          'reg_lambda' : trial.suggest_loguniform('reg_lambda', 0.001, 5.0),
          'reg_alpha' : trial.suggest_loguniform('reg_alpha', 0.001,5.0),
          'updater' : trial.suggest_categorical('updater',['shotgun','gpu_coord_descent']),                                    
           }
    if xgb_param['updater'] == 'gpu_coord_descent':
      xgb_param['feature_selector'] = trial.suggest_categorical('feature_selector',['random','greedy','thrifty']) 
      if xgb_param['feature_selector']=='greedy' or xgb_param['feature_selector']=='thrifty':
        xgb_param['top_k'] = trial.suggest_int('top_k',0,100)
    
 
    xgbr = xgb.XGBRegressor(n_jobs=-1,
                        objective='reg:squarederror',
                        booster = 'gblinear',random_state =42,
                        **xgb_param
                         )
    
    _fit_params={'early_stopping_rounds':100,
                'eval_set': [(X,y)],
                'eval_metric' : 'rmse',
                'verbose':0}
    
    scores = cross_val_score(xgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error"
                             )
    return scores.mean()


In [ ]:
gbline_param,gbline_study = tune(xgb_objective)

[I 2022-08-02 14:26:21,616] A new study created in memory with name: no-name-fcff4808-4144-4a73-b7d7-20cb4229be61
[I 2022-08-02 14:27:00,624] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 1935, 'learning_rate': 0.5754521055034162, 'reg_lambda': 0.5100627805979911, 'reg_alpha': 0.16383993835282307, 'feature_selector': 'random'}. Best is trial 0 with value: 0.0.
[I 2022-08-02 14:28:19,676] Trial 1 finished with value: -2.384185791015625e-07 and parameters: {'n_estimators': 4345, 'learning_rate': 0.05560207146770345, 'reg_lambda': 0.4160439645256604, 'reg_alpha': 0.0011916299962955155, 'feature_selector': 'random'}. Best is trial 0 with value: 0.0.
[I 2022-08-02 14:28:37,623] Trial 2 finished with value: -0.07478143945336342 and parameters: {'n_estimators': 991, 'learning_rate': 0.003407530847844932, 'reg_lambda': 0.01334697757417809, 'reg_alpha': 0.08730885649333642, 'feature_selector': 'thrifty', 'top_k': 14}. Best is trial 0 with value: 0.0.


In [ ]:
tfcv = time

In [ ]:
X = df_108.iloc[:,1:]
y = df_108['uv']
_n_trails = 100

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study

def xgb_objective(trial):
    list_bins = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250,300]
    _grow_policy = trial.suggest_categorical('grow_policy',['depthwise','lossguide']) 
    _n_estimators = trial.suggest_int('n_estimators',300,1200)
    _max_depth = trial.suggest_int('max_depth', 3, 20)    
    _max_leaves = trial.suggest_int('max_leaves',0,255)                  
    _learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.8)
    _gamma =  trial.suggest_loguniform('gamma',0.01,3.0)
    _min_child_weight =trial.suggest_categorical('min_child_weight',[1,2,3,4,5,6,7,8,9])
    _subsample = trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.1)
    _colsample_bytree=trial.suggest_discrete_uniform('colsample_bytree',0.5,1.0,0.05)
    _colsample_bylevel=trial.suggest_discrete_uniform('colsample_bylevel',0.5,1.0,0.05)    
    _colsample_bynode=trial.suggest_discrete_uniform('colsample_bynode',0.5,1.0,0.05)    
    _reg_lambda = trial.suggest_int('reg_lambda', 1, 6)
    _reg_alpha = trial.suggest_int('reg_alpha', 0, 3)
    _max_bin = trial.suggest_categorical('max_bin', list_bins)
    _sample_type =  trial.suggest_categorical('sample_type',['uniform','weighted'])
    _nomarlize_type = trial.suggest_categorical('nomarlize_type',['tree','forest'])
    _rate_drop = trial.suggest_float('rate_drop',0.0,1.0)
    _one_drop = trial.suggest_categorical('one_drop',[0,1])
    _skip_drop = trial.suggest_float('skip_drop',0.0,0.6)



    xgbr = xgb.XGBRegressor(n_jobs=-1,
                        tree_method = 'gpu_hist',
                        objective='reg:squarederror',
                        bosster = 'dart',random_state =42,
                        grow_policy = _grow_policy,
                        n_estimator = _n_estimators,
                        max_depth = _max_depth,
                        learning_rate = _learning_rate,
                        gamma = _gamma,
                        min_child_weight = _min_child_weight,
                        subsample = _subsample,
                        colsample_bytree = _colsample_bytree,
                        colsample_bylevel = _colsample_bylevel,
                        colsample_bynode = _colsample_bynode,
                        reg_lambda = _reg_lambda,
                        reg_alpha = _reg_alpha,
                        max_bin = _max_bin,
                        sample_type = _sample_type,
                        nomarlize_type = _nomarlize_type,
                        rate_drop = _rate_drop,
                        one_drop = _one_drop,
                        skip_drop = _skip_drop,
                        enable_categorical = True
                         )
    
    _fit_params={'early_stopping_rounds':100,
                'eval_set': [(X,y)],
                'eval_metric' : 'rmse',
                'verbose':0}
    
    scores = cross_val_score(xgbr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error",fit_params = _fit_params
                             )
    return scores.mean()